In [1]:
import pandas as pd
from lxml import etree
from io import BytesIO

# Load the XML file
file_path = r"C:\Users\Rodrigo\Documents\TFM\Documentos\licitacionesPerfilesContratanteCompleto3_202401\licitacionesPerfilesContratanteCompleto3.atom"
with open(file_path, 'rb') as file:  # Leer como bytes
    xml_content = file.read()

In [2]:
# Sample XML content (replace this string with your actual XML content)
# xml_content = '''
# <entry xmlns="http://www.w3.org/2005/Atom" xmlns:cac-place-ext="urn:place-ext" xmlns:cbc-place-ext="urn:place-ext">
#     <id>https://contrataciondelestado.es/sindicacion/licitacionesPerfilContratante/789966</id>
#     <link href="https://contrataciondelestado.es/wps/poc?uri=deeplink:detalle_licitacion&amp;idEvl=HwExWlQ2FHUQ0QK2TEfXGy%2BAb3%2D3" />
#     <summary type="text">Id licitación: 4270012027200; Órgano de Contratación: Jefatura de la Sección Económico-Administrativa 27 - Base Aérea de Getafe; Importe: 50377.14 EUR; Estado: RES</summary>
#     <title>Adquisición de repuestos para el ILS AMS 2100</title>
#     <updated>2013-01-11T09:47:31.139+01:00</updated>
#     <cac-place-ext:ContractFolderStatus>
#         <!-- ContractFolderStatus content here -->
#     </cac-place-ext:ContractFolderStatus>
# </entry>
# '''


parser = etree.XMLParser(ns_clean=True)
tree = etree.parse(BytesIO(xml_content), parser)  # Usar BytesIO
root = tree.getroot()

In [ ]:
# Define namespaces
ns = {
    'atom': 'http://www.w3.org/2005/Atom',
    'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
    'cac-place-ext': 'urn:place-ext',
    'cbc-place-ext': 'urn:place-ext',
}

# Initialize DataFrames
entries_df = pd.DataFrame(columns=['entry_id', 'link', 'summary', 'title', 'updated'])
contract_folder_status_df = pd.DataFrame(columns=['entry_id', 'contract_folder_id', 'contract_folder_status_code'])
# located_contracting_party_df = pd.DataFrame(columns=['contracting_party_id', 'contracting_party_type_code'])
# party_df = pd.DataFrame(columns=['party_id', 'website_uri', 'party_identification_id', 'party_identification_scheme_name', 'party_name'])
# postal_address_df = pd.DataFrame(columns=['postal_address_id', 'city_name', 'postal_zone', 'address_line', 'country_id'])
# country_df = pd.DataFrame(columns=['country_id', 'identification_code', 'identification_code_list_uri', 'name'])
# contact_df = pd.DataFrame(columns=['contact_id', 'name', 'telephone', 'telefax', 'electronic_mail'])
# procurement_project_df = pd.DataFrame(columns=['procurement_project_id', 'name', 'type_code', 'subtype_code'])
# budget_amount_df = pd.DataFrame(columns=['budget_amount_id', 'estimated_overall_contract_amount', 'total_amount', 'tax_exclusive_amount', 'currency_id'])
# required_commodity_classification_df = pd.DataFrame(columns=['required_commodity_classification_id', 'item_classification_code', 'list_uri'])
# realized_location_df = pd.DataFrame(columns=['realized_location_id', 'country_subentity', 'country_subentity_code'])
# tender_result_df = pd.DataFrame(columns=['tender_result_id', 'result_code', 'description', 'contract_issue_date'])
# winning_party_df = pd.DataFrame(columns=['winning_party_id', 'party_identification_id', 'party_identification_scheme_name', 'party_name'])
# awarded_tendered_project_df = pd.DataFrame(columns=['awarded_tendered_project_id'])
# legal_monetary_total_df = pd.DataFrame(columns=['legal_monetary_total_id', 'tax_exclusive_amount', 'payable_amount', 'currency_id'])
# tendering_terms_df = pd.DataFrame(columns=['tendering_terms_id', 'variant_constraint_indicator', 'language_id'])
# tendering_process_df = pd.DataFrame(columns=['tendering_process_id', 'procedure_code', 'urgency_code'])
# tender_submission_deadline_period_df = pd.DataFrame(columns=['tender_submission_deadline_period_id', 'end_date', 'end_time'])
# legal_document_reference_df = pd.DataFrame(columns=['legal_document_reference_id', 'document_id', 'uri', 'document_hash'])
# technical_document_reference_df = pd.DataFrame(columns=['technical_document_reference_id', 'document_id', 'uri', 'document_hash'])
# valid_notice_info_df = pd.DataFrame(columns=['valid_notice_info_id', 'notice_type_code'])
# additional_publication_status_df = pd.DataFrame(columns=['additional_publication_status_id', 'publication_media_name'])
# additional_publication_document_reference_df = pd.DataFrame(columns=['additional_publication_document_reference_id', 'issue_date'])

# Generate unique IDs for related entities
def generate_id(prefix, counter):
    return f"{prefix}_{counter}"

entry_counter = 1
contract_folder_status_counter = 1
located_contracting_party_counter = 1
party_counter = 1
postal_address_counter = 1
country_counter = 1
contact_counter = 1
procurement_project_counter = 1
budget_amount_counter = 1
required_commodity_classification_counter = 1
realized_location_counter = 1
tender_result_counter = 1
winning_party_counter = 1
awarded_tendered_project_counter = 1
legal_monetary_total_counter = 1
tendering_terms_counter = 1
tendering_process_counter = 1
tender_submission_deadline_period_counter = 1
legal_document_reference_counter = 1
technical_document_reference_counter = 1
valid_notice_info_counter = 1
additional_publication_status_counter = 1
additional_publication_document_reference_counter = 1

# Parse each entry
for entry in root.findall('atom:entry', ns):
    # Extract entry-level data
    entry_id = entry.find('atom:id', ns).text
    link_elem = entry.find('atom:link', ns)
    link = link_elem.get('href') if link_elem is not None else None
    summary = entry.find('atom:summary', ns).text
    title = entry.find('atom:title', ns).text
    updated = entry.find('atom:updated', ns).text

    # Add to entries DataFrame
    entries_df = pd.concat([entries_df, pd.DataFrame([{
        'entry_id': entry_id,
        'link': link,
        'summary': summary,
        'title': title,
        'updated': updated,
    }])], ignore_index=True)

    # Parse ContractFolderStatus
    cfs_elem = entry.find('cac-place-ext:ContractFolderStatus', ns)
    if cfs_elem is not None:
        contract_folder_id = cfs_elem.find('cbc:ContractFolderID', ns).text if cfs_elem.find('cbc:ContractFolderID', ns) is not None else None
        contract_folder_status_code_elem = cfs_elem.find('cbc-place-ext:ContractFolderStatusCode', ns)
        contract_folder_status_code = contract_folder_status_code_elem.text if contract_folder_status_code_elem is not None else None

        # Generate unique IDs
        cfs_id = generate_id('cfs', contract_folder_status_counter)
        contract_folder_status_counter += 1

        # Add to ContractFolderStatus DataFrame
        contract_folder_status_df = pd.concat([contract_folder_status_df, pd.DataFrame([{
            'entry_id': entry_id,
            'contract_folder_id': contract_folder_id,
            'contract_folder_status_code': contract_folder_status_code,
        }])], ignore_index=True)

    #     # Parse LocatedContractingParty
    #     lcp_elem = cfs_elem.find('cac-place-ext:LocatedContractingParty', ns)
    #     if lcp_elem is not None:
    #         contracting_party_type_code_elem = lcp_elem.find('cbc:ContractingPartyTypeCode', ns)
    #         contracting_party_type_code = contracting_party_type_code_elem.text if contracting_party_type_code_elem is not None else None

    #         # Generate unique IDs
    #         lcp_id = generate_id('lcp', located_contracting_party_counter)
    #         located_contracting_party_counter += 1

    #         # Add to LocatedContractingParty DataFrame
    #         located_contracting_party_df = pd.concat([located_contracting_party_df, pd.DataFrame([{
    #             'contracting_party_id': lcp_id,
    #             'contracting_party_type_code': contracting_party_type_code,
    #         }])], ignore_index=True)

    #         # Parse Party
    #         party_elem = lcp_elem.find('cac:Party', ns)
    #         if party_elem is not None:
    #             website_uri = party_elem.find('cbc:WebsiteURI', ns).text if party_elem.find('cbc:WebsiteURI', ns) is not None else None

    #             # PartyIdentification
    #             party_identification_elem = party_elem.find('cac:PartyIdentification', ns)
    #             party_identification_id_elem = party_identification_elem.find('cbc:ID', ns) if party_identification_elem is not None else None
    #             party_identification_id = party_identification_id_elem.text if party_identification_id_elem is not None else None
    #             party_identification_scheme_name = party_identification_id_elem.get('schemeName') if party_identification_id_elem is not None else None

    #             # PartyName
    #             party_name_elem = party_elem.find('cac:PartyName', ns)
    #             party_name = party_name_elem.find('cbc:Name', ns).text if party_name_elem is not None else None

    #             # Generate unique IDs
    #             party_id = generate_id('party', party_counter)
    #             party_counter += 1

    #             # Add to Party DataFrame
    #             party_df = pd.concat([party_df, pd.DataFrame([{
    #                 'party_id': party_id,
    #                 'website_uri': website_uri,
    #                 'party_identification_id': party_identification_id,
    #                 'party_identification_scheme_name': party_identification_scheme_name,
    #                 'party_name': party_name,
    #             }])], ignore_index=True)

    #             # Parse PostalAddress
    #             postal_address_elem = party_elem.find('cac:PostalAddress', ns)
    #             if postal_address_elem is not None:
    #                 city_name = postal_address_elem.find('cbc:CityName', ns).text if postal_address_elem.find('cbc:CityName', ns) is not None else None
    #                 postal_zone = postal_address_elem.find('cbc:PostalZone', ns).text if postal_address_elem.find('cbc:PostalZone', ns) is not None else None
    #                 address_line_elem = postal_address_elem.find('cac:AddressLine', ns)
    #                 address_line = address_line_elem.find('cbc:Line', ns).text if address_line_elem is not None else None

    #                 # Generate unique IDs
    #                 postal_address_id = generate_id('pa', postal_address_counter)
    #                 postal_address_counter += 1

    #                 # Add to PostalAddress DataFrame
    #                 postal_address_df = postal_address_df.append({
    #                     'postal_address_id': postal_address_id,
    #                     'city_name': city_name,
    #                     'postal_zone': postal_zone,
    #                     'address_line': address_line,
    #                     # 'country_id': country_id,  # To be set after parsing Country
    #                 }, ignore_index=True)

    #                 # Parse Country
    #                 country_elem = postal_address_elem.find('cac:Country', ns)
    #                 if country_elem is not None:
    #                     identification_code_elem = country_elem.find('cbc:IdentificationCode', ns)
    #                     identification_code = identification_code_elem.text if identification_code_elem is not None else None
    #                     identification_code_list_uri = identification_code_elem.get('listURI') if identification_code_elem is not None else None
    #                     country_name = country_elem.find('cbc:Name', ns).text if country_elem.find('cbc:Name', ns) is not None else None

    #                     # Generate unique IDs
    #                     country_id = generate_id('country', country_counter)
    #                     country_counter += 1

    #                     # Add to Country DataFrame
    #                     country_df = pd.concat([country_df, pd.DataFrame([{
    #                         'country_id': country_id,
    #                         'identification_code': identification_code,
    #                         'identification_code_list_uri': identification_code_list_uri,
    #                         'name': country_name,
    #                     }])], ignore_index=True)

    #                     # Update country_id in PostalAddress DataFrame
    #                     postal_address_df.loc[postal_address_df['postal_address_id'] == postal_address_id, 'country_id'] = country_id

    #             # Parse Contact
    #             contact_elem = party_elem.find('cac:Contact', ns)
    #             if contact_elem is not None:
    #                 contact_name = contact_elem.find('cbc:Name', ns).text if contact_elem.find('cbc:Name', ns) is not None else None
    #                 telephone = contact_elem.find('cbc:Telephone', ns).text if contact_elem.find('cbc:Telephone', ns) is not None else None
    #                 telefax = contact_elem.find('cbc:Telefax', ns).text if contact_elem.find('cbc:Telefax', ns) is not None else None
    #                 electronic_mail = contact_elem.find('cbc:ElectronicMail', ns).text if contact_elem.find('cbc:ElectronicMail', ns) is not None else None

    #                 # Generate unique IDs
    #                 contact_id = generate_id('contact', contact_counter)
    #                 contact_counter += 1

    #                 entries_df = pd.concat([entries_df, pd.DataFrame([{
    #                     'entry_id': entry_id,
    #                     'link': link,
    #                     'summary': summary,
    #                     'title': title,
    #                     'updated': updated,
    #                 }])], ignore_index=True)
entries_df

SyntaxError: prefix 'atom' not found in prefix map (<string>)

In [5]:
contract_folder_status_df

,entry_id,contract_folder_id,contract_folder_status_code
